In [1]:
import numpy as np
import matplotlib as plot
import pickle
import tarfile
import os
import requests
from collections import namedtuple
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
from IPython.utils.io import capture_output
InteractiveShell.ast_node_interactivity = "all"

In [7]:
seed = 1

# Intro
Simple notebook showing the steps to implement a NN from scratch. This notebook consists of two big parts, implementing one-layer nn and then extending it to m-layers nn. For this notebook, we are going to implement a k linear layer neural network with relu as activation. The task is image classifciation on cifar10 dataset, and the hope is to get a performance better than random guessing which given the 10 classes, should be $> 1/10$. This would indicate that it at least learnt something.

# Purpose 
There are lots of tutorials online, that are better than this notebook I'm constructing, like Karpathy's famous [micrograd series](https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ), but I wanted to take a more **analytical approach** as per my previous labs in a DL course I took years ago, showing how to go from a computational graph to deriving the math for the gradients to implementing them in a backprop. The hope is to show that this is a very mechanical process, aside from deriving some of the tricky gradients, there's a very clear pattern on how to work with this. What I had in mind should algorithmitcally be very straightforward, however the math for some of the gradients aren't trivial, and sometimes requires drawing the matrices to understand what's really happening. That's the only part I didn't find a simpler way to deal with, which I disliked, although I do think there are easier ways to work with them, something something about proper matrix calculus techniques. Admittedly, Karpathy's video is easier to understand and shorter code, since it's an implementation of automatic differentiation, whereas this implementation is purely analytical and not automatic. Regardless, it can serve as an alternative approach, to give a different perspective as it doesn't require implementing an automatic differentiation engine.

# Prerequisites
Basics of chain rule and familiarity with matrix calculus and working with matrices in general.

# Preprocess data
Originally 10000 x 32 x 32 x 3, but need to transpose it to get the same shape as in the formulas for the gradients in the derivations. There are 5 files + 1 test file and each file has 10k images. The test file has 10k as well.

* We will extract the first 10k for training and next 10k for validation. Also need to be in $d \times n$ shape, where d is the features and n the number of samples. The test data will be from the test_batch file.
* Labels should be one-hot encoding and in shape $c \times n$, where c is the classes and n the number of samples.
* Need to normalize data, subtract all data by training mean and divide by training std to turn it into zero mean unit variance data. This seems to be a [common practise](https://stats.stackexchange.com/a/202290), since the nn will be trained on the training data and therefore use those statistics. To make those statistics compatible we there need to transform the validation and test data the same way.

### Check where current folder is
We should access the cifar-10-python-tar.gz file in the data folder and extract it in the same folder. Adjust the paths accordingly if they are not correct. If the current folder path is chapter6, then it's correct.

In [ ]:
current_path = os.getcwd()
print("Current folder path:", current_path) 

In [2]:
DATA_PATH = os.path.join("data")
CIFAR_ROOT_PATH = os.path.join(DATA_PATH, "cifar-10-batches-py")
TEST_DATA_BATCH_PATH = os.path.join(CIFAR_ROOT_PATH, "test_batch")
CIFAR_TAR_PATH = os.path.join(DATA_PATH, "cifar-10-python.tar.gz")

In [6]:
if not os.path.exists(CIFAR_TAR_PATH):
    # Download cifar10, might take some time depending on internet speed, can take anywhere from few secs to a few minutes. Took me around 2mins.
    try:
        with requests.get("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", stream=True, timeout=300) as response:
            response.raise_for_status()  # Raise an error for bad status codes
            with open(CIFAR_TAR_PATH, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    with capture_output():
                        file.write(chunk)
        print("Download completed successfully.")
    except requests.exceptions.Timeout:
        print("The request timed out.")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        
if not os.path.exists(CIFAR_ROOT_PATH):
    with tarfile.open(CIFAR_TAR_PATH) as tar:
        tar.extractall(path=DATA_PATH)

In [ ]:
# utils for preprocessing data

Data = namedtuple('Data', 'x y')

def load_data(file_path):
    with open(file_path, 'rb') as file:
        dict = pickle.load(file, encoding='bytes')
    return dict[b'data'], dict[b'labels']

def load_cifar10(number_of_batches):
    assert 0 < number_of_batches < 6
    data = []
    labels = []
    test_data, test_labels = load_data(TEST_DATA_BATCH_PATH)
    for i in range(1, number_of_batches + 1):
        d, l = load_data(os.path.join(CIFAR_ROOT_PATH, f"data_batch_{i}"))
        data.append(d)
        labels.append(l)
    # in data rows are samples so stack them on the rows with vstack
    # in labels columns are samples so stack them on the columns with hstack
    return np.vstack(data), np.hstack(labels), np.array(test_data), np.array(test_labels)

def train_valid_split(data, labels, split_ratio):
    assert type(split_ratio) is list and len(split_ratio) == 2 
    x_train, x_valid, y_train, y_valid = train_test_split(data, labels, train_size=split_ratio[0], test_size=split_ratio[1], shuffle=False)
    return x_train, y_train, x_valid, y_valid
    
def transform_data(x_train, x_valid, x_test, y_train, y_valid, y_test):
    # normalize data
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    x_train = (x_train - mean) / std
    x_valid = (x_valid - mean) / std
    x_test = (x_test - mean) / std
    assert np.allclose(np.mean(x_train, axis=0), np.zeros(x_train.shape[1]))
    assert np.allclose(np.std(x_train, axis=0), np.ones(x_train.shape[1]))
    
    # transform data to d x n to adhere to the gradient formulas
    x_train = x_train.T
    x_valid = x_valid.T
    x_test = x_test.T
    
    # transform labels to one-hot encoding
    y_train_one_hot = np.zeros((10, y_train.size))
    y_train_one_hot[y_train, np.arange(y_train.size)] = 1
    y_valid_one_hot = np.zeros((10, y_valid.size))
    y_valid_one_hot[y_valid, np.arange(y_valid.size)] = 1
    y_test_one_hot = np.zeros((10, y_test.size))
    y_test_one_hot[y_test, np.arange(y_test.size)] = 1
    
    return Data(x_train, y_train_one_hot), Data(x_valid, y_valid_one_hot), Data(x_test, y_test_one_hot)


In [ ]:
# Only load the first 20k data, first 10k is for training and other 10k is valid, test batch is a separate data with 10k
# data is wrapped in a Data namedtuple with field x for the data and y for the labels
x_data, y_data, x_test, y_test =  load_cifar10(2)
assert x_data.shape == (20000, 3072)
assert y_data.shape == (20000,)
assert x_test.shape == (10000, 3072)
assert y_test.shape == (10000,)

x_train, y_train, x_valid, y_valid = train_valid_split(x_data, y_data, [0.5, 0.5])
assert x_train.shape == (10000, 3072)
assert x_valid.shape == (10000, 3072)
assert y_train.shape == (10000,)
assert y_valid.shape == (10000,)

train, valid, test = transform_data(x_train, x_valid, x_test, y_train, y_valid, y_test)
assert train.x.shape == (3072, 10000)
assert valid.x.shape == (3072, 10000)
assert test.x.shape == (3072, 10000)
assert train.y.shape == (10, 10000)
assert valid.y.shape == (10, 10000)
assert test.y.shape == (10, 10000)

# Deriving the gradients for the NN

We will start with a two-layer nn and extend it to k-layer nn. This is because the gradient of the k-layer nn is almost the same as two-layer except that it will need a loop to continuously update the layers, although using the same formulas as a two-layer nn. **Once again, note that we will consider a single data point and then extend to batches of data.** The computational graph we have for this 2-layer network is illustrated below:

![](./assets/two_layer_nn_comp_graph.png)

where $J$ is the cost function, which takes the output from the final layer to compute the loss. Our functions are thus

$$J = l + \lambda r$$

$$r = \|w_1\|^2 + \|w_2\|$$

$$l = -\log(y^T p)$$

$$p = \operatorname{softmax}(s_2)$$

$$s_2 = w_2h + b_2$$

$$h = \max(0, s_1)$$

$$s_1 = w_1x + b_1$$

**Dimensions.** Note that 
* $y$ is a one-hot encoding, consisting of zeros except for one element that is 1, of size $c \times 1$ for a single data point and $c \times n$ for a dataset of size n. c stands for number of classes, in cifar10 that's 10 classes.
* $w_1$ is $k \times d$, where d is the number of features of the data and k the features of the matrix.
* $x$ is $d \times 1$ for single data point and $d \times n$ for a batch of size n.
* $s_1$ is $k \times 1$ and $k \times n$.
* $h$ is $k \times 1$ and $k \times n$.
* $s_2$ is $c \times 1$ and $c \times n$.
* $p$ is $c \times 1$ and $c \times n$ for a batch.
* $l$ and $J$ both are scalars because they are loss functions.
* $r$ is a scalar, because it's the frobenius norm (l2-norm) of the matrix $w_1$ and $w_2$.

We want to find the optimal parameters for the one linear layer neural network. This nn has $w_1$, $b_1$, $w_2$, $b_2$ as model parameters, so these are the gradients we are looking for

$$\frac{\partial J}{\partial w_1},\  \frac{\partial J}{\partial w_2},\  \frac{\partial J}{\partial b_1},\  \frac{\partial J}{\partial b_2}$$

Following the computational graph, the backprop starts from the end and propagates to the start. Applying the chain rule on J w.r.t w we therefore get when we go backwards from J to $s_2$ and $s_1$:

$$\frac{\partial J}{\partial w_2} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \frac{\partial s_2}{\partial w_2} + \frac{\partial J}{\partial r} \frac{\partial r}{\partial w_2}$$

$$\frac{\partial J}{\partial w_1} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1} \frac{\partial s_1}{\partial w_1} + \frac{\partial J}{\partial r} \frac{\partial r}{\partial w_1}$$

similarily we get for J w.r.t $b_1$ and $b_2$:
$$\frac{\partial J}{\partial b_2} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \frac{\partial s_2}{\partial b_2}$$

$$\frac{\partial J}{\partial b_1} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1} \frac{\partial s_1}{\partial b_1}$$

where $g^T = -(y-p)^T$.
I've highlighted the red part that occurs in the gradients. In the code we can be more efficient and store this as a variable to avoid recomputing it each time.

We start with one single data point and later extend it to a batch of data. Moreover, we use the [numerator layout](https://en.wikipedia.org/wiki/Matrix_calculus#Numerator-layout_notation) when computing the gradients. For clarification here's a simple example that should illustrate the idea of the numerator layout

$$
\frac{\partial f}{\partial x} =
\begin{bmatrix}
\frac{\partial f_{1}}{\partial x_{1}} \dots{} \frac{\partial f_{1}}{\partial x_{n}} \\
\vdots \\
\frac{\partial f_{m}}{\partial x_{1}} \dots{} \frac{\partial f_{m}}{\partial x_{n}}
\end{bmatrix}
$$

For row 1 we fix f and vary x. For row 2 we fix the next element in f, which is $f_{2}$ and vary x the same way. We do this all the way down to the last row m, which is we fix $f_m$ and vary x because there are only m outputs from the vector-valued function $f: \mathbb{R}^n \to \mathbb{R}^m$. This produces a $m \times n$ jacobian matrix that is the derivative of the vector-valued function $f$ w.r.t the vector $x$.
This pattern naturally applies to gradients and scalar functions with scalar inputs as well. So really it's a general method that works for all kinds of situations, as long as the derivative exists.


## Computing $\frac{\partial J}{\partial b}$
The gradient of J w.r.t b is the shortest so we start with it. Recall that we had 

$$\frac{\partial J}{\partial b_2} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \frac{\partial s}{\partial b_2}$$

We can reuse alot of insights and results of the gradients from the previous notebook with one-layer nn. We know from the previous notebook that 

$$\frac{\partial s_2}{\partial b_2} = I_c$$

so we have 

$$\frac{\partial J}{\partial b_2} = g^T$$

and for 

$$\frac{\partial J}{\partial b_1} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \textcolor{green}{\frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1}} \frac{\partial s_1}{\partial b_1}$$

we only need to look at $\frac{\partial s_2}{\partial h}$ and $\frac{\partial h}{\partial s_1}$. Recall that $s_2 = w_2h + b_2$ so we get

$$\frac{\partial s_2}{\partial h} = w_2$$

and that $h = \max(0, s_1)$ so that the derivative of relu is

$$
\begin{cases}
1_i & s_{1}^{(i)} > 0 \\
0 & s_{1}^{(i)} \leq 0
\end{cases}
$$

writing this out in numerator layout

$$
\frac{\partial h}{\partial s_1} = 
\begin{bmatrix}
\frac{\partial h_1}{\partial s_{11}} \dots{} \frac{\partial h_{1}}{\partial s_{1k}} \\
\vdots \\
\frac{\partial h_{k}}{\partial s_{11}} \dots{} \frac{\partial h_{k}}{\partial s_{1k}}
\end{bmatrix} =
\begin{bmatrix}
\operatorname{Ind}(s_{11}) & 0 & \cdots & 0 \\
0 & \operatorname{Ind}(s_{12}) & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & \operatorname{Ind}(s_{1k})
\end{bmatrix} =
\operatorname{diag}(\operatorname{Ind}(s_1 > 0))
$$

where $\operatorname{Ind}$ is the indicator function that returns 1 if the conditional inside it is true.
Therefore we get 

$$\frac{\partial J}{\partial b_1} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \textcolor{green}{\frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1}} \frac{\partial s_1}{\partial b_1} = g^T w_2 \operatorname{diag}(\operatorname{Ind}(s_1 > 0))$$


### Putting $\frac{\partial J}{\partial b}$ together

$$\frac{\partial J}{\partial b_2} = g^T$$
$$\frac{\partial J}{\partial b_1} = g^T w_2 \operatorname{diag}(\operatorname{Ind}(s_1 > 0))$$

Note that if we assign $g^T \leftarrow g^T w_2 \operatorname{diag}(\operatorname{Ind}(s_1 > 0))$, then we can reuse it in $\frac{\partial J}{\partial b_1}$ and $\frac{\partial J}{\partial w_1}$ making the math a little bit less cluttery.

## Computing $\frac{\partial J}{\partial w}$

Recall we had the chain of gradients 

$$\frac{\partial J}{\partial w_2} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \frac{\partial s_2}{\partial w_2} + \textcolor{orange}{\frac{\partial J}{\partial r} \frac{\partial r}{\partial w_2}}$$

$$\frac{\partial J}{\partial w_1} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \textcolor{green}{\frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1}} \frac{\partial s_1}{\partial w_1} + \textcolor{orange}{\frac{\partial J}{\partial r} \frac{\partial r}{\partial w_1}}$$


### $\textcolor{orange}{\frac{\partial J}{\partial r} \frac{\partial r}{\partial w}}$

The functions are $J = l + \lambda r$ and $r = \|w_1\|^2 + \|w_2\|^2$ and it's easy to see that we get 

$$\frac{\partial J}{\partial r} \frac{\partial r}{\partial w_2} = \lambda 2w_2$$
$$\frac{\partial J}{\partial r} \frac{\partial r}{\partial w_1} = \lambda 2w_1$$

### $\textcolor{green}{\frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1}}$

We computed this in the previous section

$$\textcolor{green}{\frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1}} = w_2 \operatorname{diag}(\operatorname{Ind}(s_1 > 0))$$

### Putting together $\frac{\partial J}{\partial w}$

$J$ w.r.t $w_2$ is exactly like in the one-layer case except we have $h$ instead of $x$, so we can reuse the results from the previous notebook

$$\frac{\partial J}{\partial w_2} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}}}} \frac{\partial s_2}{\partial w_2} + \textcolor{orange}{\frac{\partial J}{\partial r} \frac{\partial r}{\partial w_2}} = 2\lambda + gh^T$$

As for $J$ w.r.t $w_1$ the result is almost exactly the same. If we assign $g^T \leftarrow g^T w_2 \operatorname{diag}(\operatorname{Ind(s_1 > 0)})$ from $\frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1}$, then we will get the exact same, only having to remember that $g^T$ has changed value because of the assignment

$$\frac{\partial J}{\partial w_1} = \underset{g^T}{\underbrace{\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}} \frac{\partial s_2}{\partial h} \frac{\partial h}{\partial s_1}}} \frac{\partial s_1}{\partial w_1} + \textcolor{orange}{\frac{\partial J}{\partial r} \frac{\partial r}{\partial w_1}} = 2\lambda w_1 + g x^T$$


## Extending to batches of data

Remember that the gradients computed

$$\frac{\partial J}{\partial w_2} = gh^T + 2\lambda w_2$$
$$\frac{\partial J}{\partial b_2} = -(y-p)^T = g^T$$

If we make this assignment to $g^T \leftarrow g^T w_2 \operatorname{diag}(\operatorname{Ind}(s_1 > 0))$, then we can reuse it like we would do in the code. 
$$\frac{\partial J}{\partial w_1} = gx^T + 2\lambda w_1$$
$$\frac{\partial J}{\partial b_1} = g^T$$

These were for one single data point. However, we can easily extend this to a batch of data of arbitrary size. Assume $x_b$ is of size $d \times n$ and one-hot encoding labels $y_b$ is size $c \times n$, where n is the size of the data. Then in the forward pass we have

$$h_b = \max(wx_b + b_1 1^T_n, 0)$$
$$p_b = \operatorname{softmax}(w_2h_b + b_2 1^T_n)$$

where 
* $x_b$ is $d \times n$
* $p_b$ is $c \times n$
* $w_2$ is $c \times k$
* $b_2$ is $c \times 1$
* $b_2 1^T_n$ is $c \times n$ and $1^T_n$ is $1 \times n$ which is used to broadcast of the bias vector $b_2$ where it's duplicated for each data point
* $w_1$ is $k \times d$
* $b_1$ is $d \times 1$
* $b_1 1^T_n$ is $k \times n$ with $1^T_n$ being $1 \times n$, broadcasting in the same way as $b_2$

In the backward pass we have a natural extension of the gradients we already computed, much in the same way as we showed in the previous notebook

$$g_b = -(y_b - p_b)$$

$$\frac{\partial J}{\partial w_2} = \frac{1}{n} g_b h_b^T + 2\lambda w_2$$
$$\frac{\partial J}{\partial b_2} = \frac{1}{n} g_b 1_n$$

We want to assign $g^T \leftarrow g^T w_2 \operatorname{diag}(\operatorname{Ind}(s_1 > 0))$ so we can reuse $g^T$, however with batches it changes to $g^T \leftarrow (w_2^T g_b) \odot \operatorname{Ind}(s_1^{(b)} > 0)$, which is not obvious. We can understand this when we look at the **effect** of the case when only one data point was used. 

$$
g^T \operatorname{diag}(\operatorname{Ind}(s_1 > 0)) = (g1, \dots, g_c)
\begin{bmatrix}
\operatorname{Ind}(s_{11} > 0) & 0 & \dots & 0 \\
0 & \operatorname{Ind}(s_{12} > 0) & \dots & 0 \\
\vdots & \ddots & \vdots \\
0 & \dots & \operatorname{Ind}(s_{1k} > 0)
\end{bmatrix}
= [g_1 \operatorname{Ind}(s_{11} > 0), g_2 \operatorname{Ind}(s_{12} > 0), \dots, g_k\operatorname{Ind}(s_{1k} > 0)] = g \cdot \operatorname{Ind}(s_1 > 0)
$$

We can see that the effect is elementwise multiplication between g and the diagonals of $\operatorname{diag}(\operatorname{Ind}(s_1 > 0))$, but with a data batch we have a $h_b$ matrix of size $k \times n$ instead, where each column constitutes a sample. To achieve a elementwise multiplication of each corresponding samples (we match the samples and perform elementwise multiplication between them), meaning column 1 of $g_b$ dot product column 1 of $h_b$, column 2 of $g_b$ dot product column 2 of $h_b$ etc. This constitutes a hadamard product between the two, which is what was stated earlier. Finally after the assignment $g^T \leftarrow (w_2^T g_b) \odot \operatorname{Ind}(s_1^{(b)} > 0)$ we get

$$\frac{\partial J}{\partial w_1} = \frac{1}{n} g_b x_b^T + 2\lambda w_1$$
$$\frac{\partial J}{\partial b_1} = \frac{1}{n} g_b 1_n$$




# Extending to batches of data for k-layer nn
The gradients are exactly the same except they iteratively change during the loop. The only two things to note
* For the forward pass, the last iteration should not rewrite the relevant variables because softmax is applied at the end, so the iterations need to end prematurely by one iteration. 
* For the backward pass, the iterations goes backwards and should also end prematurely by one iteration. However in this case you can just put an conditional that exits at the last iteration, then don't need to duplicate code.

Here's the pseudocode for it:

**Forward pass**
```py
# Let X_batch^(0) = X_batch

for i in range(1, k):  # i = 1, ..., k-1
    X_batch[i] = max(W[i] @ X_batch[i-1] + b[i] * ones(n_b).T, 0)

# Compute final probabilities
P_batch = SoftMax(W[k] @ X_batch[k-1] + b[k] * ones(n_b).T)
```

**Backward pass**
```py
# Step 1: Initialize gradient for the output layer
G_batch = -(Y_batch - P_batch)

# Step 2: Loop through layers in reverse (from k to 2)
for l in range(k, 1, -1):
    # Compute gradients with respect to weights and biases
    dL_dW[l] = (1 / n_b) * G_batch @ X_batch[l-1].T
    dL_db[l] = (1 / n_b) * G_batch @ ones(n_b)
    
    # Backpropagate the gradient through the layer
    G_batch = W[l].T @ G_batch
    G_batch = G_batch * (X_batch[l-1] > 0)  # Element-wise multiplication with indicator function

# Step 3: Compute gradients for the first layer
dL_dW[1] = (1 / n_b) * G_batch @ X_batch.T
dL_db[1] = (1 / n_b) * G_batch @ ones(n_b)

```


